### Fake News Project

By: Mateo Anusic, Emil Thorlund, Lucas A. Rosing, Victor Bergien

### Task #1

- Tokenize the text
- Remove stopwords and compute the size of the vocabulary.
- Compute the reduction rate of the vocabulary size after removing stopwords.
- Remove word variations with stemming and compute the size of the vocabulary.
- Compute the reduction rate of the vocabulary size after stemming.

Describe which procedures (and which libraries) you used and why they are appropriate.

In [2]:
# Importing the libraries

import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import pandas as pd
import csv

# Importing the dataset
data = pd.read_csv('/Volumes/Glyph1TB/newsCorpus/data_3GB.csv')

# Prints the first 5 rows of the dataset for a quick look
data.head()

,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source,processed_text
0,0,2,express.co.uk,rumor,https://www.express.co.uk/news/science/738402/...,"life illus , least quantum level , theori rece...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,life illus ? research prove 'realiti n't exist...,Sean Martin,NaN,[''],"univers ceas exist look prove life illus , acc...",NaN,NaN,NaN,NaN
1,1,6,< url >,hate,http://barenakedislam.com/category/donald-trum...,"unfortun , ’ yet attack islam terrorism-connec...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,donald trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",NaN,[''],NaN,NaN,NaN,NaN,NaN
2,2,7,< url >,hate,http://barenakedislam.com/category/donald-trum...,lo angel polic depart deni $ < num > million f...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,donald trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",NaN,[''],NaN,NaN,NaN,NaN,NaN
3,3,8,< url >,hate,http://barenakedislam.com/2017/12/24/more-winn...,white hous decid quietli withdraw tie ramallah...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"win ! isra intellig sourc , debkafil , confirm...","Cleavis Nowell, Cleavisnowell, Clarence J. Fei...",NaN,[''],NaN,NaN,NaN,NaN,NaN
4,4,9,< url >,hate,http://barenakedislam.com/2017/12/25/oh-trump-...,“ time come cut tongu support peac mourn it. ”...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"“ oh , trump , coward , wait , dig grave mean ...","F.N. Lehner, Don Spilman, Clarence J. Feinour,...",NaN,[''],NaN,NaN,NaN,NaN,NaN


In [3]:
data2 = data.drop(columns = ["Unnamed: 0", "meta_description", "keywords", "tags", "summary", "meta_keywords", "authors", "updated_at",  "scraped_at",  "inserted_at", "url","id", "domain", "source", "processed_text"], axis=1)

In [4]:
print(data2.head())


    type                                            content  \
0  rumor  life illus , least quantum level , theori rece...   
1   hate  unfortun , ’ yet attack islam terrorism-connec...   
2   hate  lo angel polic depart deni $ < num > million f...   
3   hate  white hous decid quietli withdraw tie ramallah...   
4   hate  “ time come cut tongu support peac mourn it. ”...   

                                               title  
0  life illus ? research prove 'realiti n't exist...  
1                                       donald trump  
2                                       donald trump  
3  win ! isra intellig sourc , debkafil , confirm...  
4  “ oh , trump , coward , wait , dig grave mean ...  


In [5]:
# Creates a subset of the dataset with only the columns we need
data4 = data2.dropna(subset=['content', 'title', 'type'])

# Prints the first 5 rows of the dataset for a quick look
print(data4) 

# Removes the rows with unknown or None values
data4 = data4[(data4 != 'unknown') & (data4 != 'None')].dropna()

        type                                            content  \
0      rumor  life illus , least quantum level , theori rece...   
1       hate  unfortun , ’ yet attack islam terrorism-connec...   
2       hate  lo angel polic depart deni $ < num > million f...   
3       hate  white hous decid quietli withdraw tie ramallah...   
4       hate  “ time come cut tongu support peac mourn it. ”...   
...      ...                                                ...   
71453   fake  uruguay bishop defend pro-lif coupl headlin : ...   
71454   fake  someth els going-on mani us specul help ' q-an...   
71455   fake  headlin : bitcoin & blockchain search exceed t...   
71456   fake  someth els going-on mani us specul help ' q-an...   
71457   fake  headlin : bitcoin & blockchain search exceed t...   

                                                   title  
0      life illus ? research prove 'realiti n't exist...  
1                                           donald trump  
2                 

In [6]:
# Creates a function which simplifies the type column
# This allows for an easier classification
def modify_type(x):
    if x == 'fake':
        return 0
    elif x == 'reliabl':
        return 1
    elif x == 'rumor':
        return 2
    elif x == 'satir':
        return 3
    elif x == 'bia':
        return 4
    elif x == 'conspiraci':
        return 5
    elif x == 'junksci':
        return 6
    elif x == 'hate':
        return 7
    elif x == 'clickbait':
        return 8
    elif x == 'unreli':
        return 9
    elif x == 'polit':
        return 10
    else:
        return None   

"\ndef modify_type(x):\n    if x == 'fake':\n        return 0\n    elif x == 'reliable':\n        return 1\n    elif x == 'rumor':\n        return 2\n    elif x == 'satire':\n        return 3\n    elif x == 'bias':\n        return 4\n    elif x == 'conspiracy':\n        return 5\n    elif x == 'junksci':\n        return 6\n    elif x == 'hate':\n        return 7\n    elif x == 'clickbait':\n        return 8\n    elif x == 'unreliable':\n        return 9\n    elif x == 'political':\n        return 10\n    else:\n        return None    \n"

In [7]:
# Creates a new dataframe with the modified type column
data5 = pd.DataFrame(data4)

# Writes type column to the new dataframe
data5['type'] = data5['type'].apply(modify_type)

# Prints the dataframe to verify the changes
data5 

,type,content,title
0,2,"life illus , least quantum level , theori rece...",life illus ? research prove 'realiti n't exist...
1,7,"unfortun , ’ yet attack islam terrorism-connec...",donald trump
2,7,lo angel polic depart deni $ < num > million f...,donald trump
3,7,white hous decid quietli withdraw tie ramallah...,"win ! isra intellig sourc , debkafil , confirm..."
4,7,“ time come cut tongu support peac mourn it. ”...,"“ oh , trump , coward , wait , dig grave mean ..."
...,...,...,...
71453,0,uruguay bishop defend pro-lif coupl headlin : ...,uruguay bishop defend pro-lif coupl
71454,0,someth els going-on mani us specul help ' q-an...,исполнительный директор фонда святителя григор...
71455,0,headlin : bitcoin & blockchain search exceed t...,ef pontif mass london exalt cross
71456,0,someth els going-on mani us specul help ' q-an...,χοροστασια του πατριαρχικου επιτροπου καλλιουπ...


In [8]:
# Creates a new dataframe in order to check that there are no 'none' values
data6 = data2.dropna(subset=['type'])

In [9]:
# Import the necessary libraries
import numpy as np

In [10]:
# Sorts the dataframe by the type column
data_by_type = {}

# A for-loop which creates a new dataframe for each type
for i in range(11):
    data_by_type[i] = data5[data5['type'] == i]

data_zero = data_by_type[0]
data_one = data_by_type[1]
data_two = data_by_type[2]
data_three = data_by_type[3]
data_four = data_by_type[4]
data_five = data_by_type[5]
data_six = data_by_type[6]
data_seven = data_by_type[7]
data_eight = data_by_type[8]
data_nine = data_by_type[9]
data_ten = data_by_type[10]


In [11]:
# Prints the dataframe containing only type 1 data
data_one

,type,content,title
42,1,facebook/destinythegam promo imag `` destini <...,'destini < num > ' news : bungi address player...
83,1,( screenshot : facebook ) dr. melina abdullah ...,professor conduct libat summon spirit black ki...
84,1,( photo : reuter ) team north south korea marc...,'ghost ship ' < num > dead n. korean wash japa...
85,1,( photo : reuters/dado ruvic ) facebook . upda...,facebook lift ban paid ad pro-lif film produc ...
86,1,( screencap : youtube/tidningen dagen ) annahi...,iranian christian convert lead < num > muslim ...
...,...,...,...
33750,1,"congoles rebel group , < num > , seri victori ...",congo ’ troubl futur
33752,1,ironi modern polit mani conserv american view ...,biblic econom
33760,1,"terror attack , rush identifi ’ blame analyz s...",bangladesh
44067,1,( photo : reuters/raheb homavandi ) christian ...,iranian christian hit back bishop claim 'not p...


In [12]:
# Create DataFrame for fake data with only zeros in "type" column
data_zero_only = pd.DataFrame({'type': np.zeros(len(data_zero))})
data_one_only = pd.DataFrame({'type': np.ones(len(data_one))})
data_two_only = pd.DataFrame({'type': 2 * np.ones(len(data_two))})
data_three_only = pd.DataFrame({'type': 3 * np.ones(len(data_three))})
data_four_only = pd.DataFrame({'type': 4 * np.ones(len(data_four))})
data_five_only = pd.DataFrame({'type': 5 * np.ones(len(data_five))})
data_six_only = pd.DataFrame({'type': 6 * np.ones(len(data_six))})
data_seven_only = pd.DataFrame({'type': 7 * np.ones(len(data_seven))})
data_eight_only = pd.DataFrame({'type': 8 * np.ones(len(data_eight))})
data_nine_only = pd.DataFrame({'type': 9 * np.ones(len(data_nine))})
data_ten_only = pd.DataFrame({'type': 10 * np.ones(len(data_ten))})



In [13]:
# Prints the dataframe containing only type 5 data
data_five_only

,type
0,5.0
1,5.0
2,5.0
3,5.0
4,5.0
...,...
5495,5.0
5496,5.0
5497,5.0
5498,5.0


In [14]:
# Creates the datasets with type, content and title columns

zero_dataset = data_zero[['type', 'content', 'title']]
one_dataset = data_one[['type', 'content', 'title']]
two_dataset = data_two[['type', 'content', 'title']]
three_dataset = data_three[['type', 'content', 'title']]
four_dataset = data_four[['type', 'content', 'title']]
five_dataset = data_five[['type', 'content', 'title']] 
six_dataset = data_six[['type', 'content', 'title']] 
seven_dataset = data_seven[['type', 'content', 'title']] 
eight_dataset = data_eight[['type', 'content', 'title']] 
nine_dataset = data_nine[['type', 'content', 'title']] 
ten_dataset = data_ten[['type', 'content', 'title']]

five_dataset 

,type,content,title
10,5,rev dr. childress avail come church event ! de...,blackgenocide.org
11,5,speak engag request contact person name : * ph...,request speak engag
12,5,"`` …i set life death , bless curs ’ therefor c...",blackgenocide.org
13,5,oppos plan parent hood ( follow reprint stopp ...,oppos plan parenthood
56,5,love children sayso march < num >,sayso march < num >
...,...,...,...
70211,5,'s busi usual washington . presid obama sign a...,obama dictat nation `` cyber secur '' emerg
70310,5,"fallen < num > straight day , crush hope dream...","today , baltic dri freight index never lower"
70311,5,baltic dri index - admir soar support thing gr...,baltic dri index < num > % year-to-d ; worst r...
70355,5,"< url > april < num > , < num > friday , april...",glenn beck don freemason patch broadcast


In [15]:
# Concatenates the datasets into one dataset
data7 = pd.concat([zero_dataset,
                    one_dataset,
                    two_dataset,
                    three_dataset,
                    four_dataset,
                    five_dataset,
                    six_dataset,
                    seven_dataset,
                    eight_dataset,
                    nine_dataset,
                    ten_dataset
                    ], axis=0)

# Prints the new dataset
data7

,type,content,title
27,0,headlin : bitcoin & blockchain search exceed t...,surpris : socialist hotb venezuela lost half ’...
28,0,water cooler < num > / < num > / < num > open ...,water cooler < num > / < num > / < num > open ...
29,0,veteran comment call grow “ ethnonation ” fox ...,veteran comment call grow “ ethnonation ” fox ...
30,0,"lost word , hidden word , otter , bank book % ...","lost word , hidden word , otter , bank book"
31,0,red alert : bond yield scream “ inflat come ! ...,red alert : bond yield scream “ inflat come ! ”
...,...,...,...
71447,10,"florida 's invad : flamingo pink flamingo , co...",floridageorg 's stream
71448,10,"choic optim dark . `` , '' `` stronger togeth ...",post histori floridageorg
71450,10,"absurd race continu rear ugli head , succubu a...",absurd racialist playbook
71451,10,"erstwhil boss , legendari crisis-manag guru li...",bp : beyond pr


In [16]:
# Removes the rows with unknown or None values

data7.loc[data5['content'] == " ", 'content']= np.nan
data7.loc[data5['content'] == "  ", 'content']= np.nan

In [17]:
# Removes the rows with unknown or None values
data8 = data7.dropna()

In [18]:
# Prints the new dataset
data8.head()

,type,content,title
0,2,"life illus , least quantum level , theori rece...",life illus ? research prove 'realiti n't exist...
1,7,"unfortun , ’ yet attack islam terrorism-connec...",donald trump
2,7,lo angel polic depart deni $ < num > million f...,donald trump
3,7,white hous decid quietli withdraw tie ramallah...,"win ! isra intellig sourc , debkafil , confirm..."
4,7,“ time come cut tongu support peac mourn it. ”...,"“ oh , trump , coward , wait , dig grave mean ..."


In [19]:
# Renames the content column to con
data9 = data8.rename(columns = {"content":"con"})

In [20]:
# Prints the new dataset
data9

,type,con,title
0,2,"life illus , least quantum level , theori rece...",life illus ? research prove 'realiti n't exist...
1,7,"unfortun , ’ yet attack islam terrorism-connec...",donald trump
2,7,lo angel polic depart deni $ < num > million f...,donald trump
3,7,white hous decid quietli withdraw tie ramallah...,"win ! isra intellig sourc , debkafil , confirm..."
4,7,“ time come cut tongu support peac mourn it. ”...,"“ oh , trump , coward , wait , dig grave mean ..."
...,...,...,...
71453,0,uruguay bishop defend pro-lif coupl headlin : ...,uruguay bishop defend pro-lif coupl
71454,0,someth els going-on mani us specul help ' q-an...,исполнительный директор фонда святителя григор...
71455,0,headlin : bitcoin & blockchain search exceed t...,ef pontif mass london exalt cross
71456,0,someth els going-on mani us specul help ' q-an...,χοροστασια του πατριαρχικου επιτροπου καλλιουπ...


In [21]:
# Creates a new dataframe with the modified type column
data10 = data9["con"]

In [22]:
# Squeezes the dataframe
sqeeze = data9.squeeze(1)


In [23]:
# Makes sure that all content rows are less than 1000 characters
data10 = data8["content"].str.slice(start = 0, stop = 1000)

In [24]:
# Creates a new dataframe which is concatenated with the type column
data11 = pd.concat([sqeeze,data10], axis=1).drop(columns='con')

# Prints the new dataset
data11

,type,title,content
0,2,life illus ? research prove 'realiti n't exist...,"life illus , least quantum level , theori rece..."
1,7,donald trump,"unfortun , ’ yet attack islam terrorism-connec..."
2,7,donald trump,lo angel polic depart deni $ < num > million f...
3,7,"win ! isra intellig sourc , debkafil , confirm...",white hous decid quietli withdraw tie ramallah...
4,7,"“ oh , trump , coward , wait , dig grave mean ...",“ time come cut tongu support peac mourn it. ”...
...,...,...,...
71453,0,uruguay bishop defend pro-lif coupl,uruguay bishop defend pro-lif coupl headlin : ...
71454,0,исполнительный директор фонда святителя григор...,someth els going-on mani us specul help ' q-an...
71455,0,ef pontif mass london exalt cross,headlin : bitcoin & blockchain search exceed t...
71456,0,χοροστασια του πατριαρχικου επιτροπου καλλιουπ...,someth els going-on mani us specul help ' q-an...


In [25]:
# Creates the X and y variables which will be used for the classification
x = data11.drop(columns="type") 
y = data11["type"]

In [26]:
# Prints the X variable
x

,title,content
0,life illus ? research prove 'realiti n't exist...,"life illus , least quantum level , theori rece..."
1,donald trump,"unfortun , ’ yet attack islam terrorism-connec..."
2,donald trump,lo angel polic depart deni $ < num > million f...
3,"win ! isra intellig sourc , debkafil , confirm...",white hous decid quietli withdraw tie ramallah...
4,"“ oh , trump , coward , wait , dig grave mean ...",“ time come cut tongu support peac mourn it. ”...
...,...,...
71453,uruguay bishop defend pro-lif coupl,uruguay bishop defend pro-lif coupl headlin : ...
71454,исполнительный директор фонда святителя григор...,someth els going-on mani us specul help ' q-an...
71455,ef pontif mass london exalt cross,headlin : bitcoin & blockchain search exceed t...
71456,χοροστασια του πατριαρχικου επιτροπου καλλιουπ...,someth els going-on mani us specul help ' q-an...


In [27]:
# Prints the y variable
y

0        2
1        7
2        7
3        7
4        7
        ..
71453    0
71454    0
71455    0
71456    0
71457    0
Name: type, Length: 67334, dtype: int64

In [28]:
# Creates csv files for the X and y variables
x.to_csv("features_multi.csv", index = False)

y.to_csv("labels_multi.csv", index = False)